In [1]:
from clpsych.store import Store
from clpsych.data import *

import codecs
import os

In [9]:
tokenized = {}
i = 0
base_i = i
more = True
while more:
    if i % 10 == 0: print i
    if os.path.isfile('data/parses/{}.parse'.format(i)):
        docs, titles = read_parses('data/parses/{}.parse'.format(i))
        keys = [key for key in docs.keys() if key]
        for key in keys:
            doc, title = [], []
            for token in docs[key]:
                doc.append(token.lemma)
            for token in titles[key]:
                title.append(token.lemma)
            tokenized[key] = (title, doc)
        i += 1
    else:
        break

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210


# Labeling Data

In [2]:
def line(post_id, title, doc):
    return post_id + '\t' + ' '.join(title) + '\t' + ' '.join(doc) + '\n'

def save_classes(dataframe, filename='post_class.txt'):
    sub = dataframe[['post_id', 'user_id']]
    sub['class'] = sub['user_id'] > 0
    sub[['post_id', 'class']].to_csv(filename)

In [3]:
store = Store('data.h5')

In [4]:
df = store.data

In [9]:
# remove posts from suicide_watch
df = df[df.subreddit != 'SuicideWatch']

In [10]:
train = df.merge(store.train_indices, on='user_id')
test = df.merge(store.test_indices, on='user_id')
dev = df.merge(store.dev_indices, on='user_id')
sample = df.merge(store.sample_indices, on='user_id')

In [11]:
save_classes(train, 'data/classes/train_classes.txt')
save_classes(test, 'data/classes/test_classes.txt')
save_classes(dev, 'data/classes/dev_classes.txt')
save_classes(sample, 'data/classes/sample_classes.txt')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Generating Sub-pieces of the Parse

In [12]:
keys = [key for key in tokenized.keys() if key]

with codecs.open('data/tokens/lemmas.txt', 'w', encoding='utf-8') as fp:
    for key in keys:
        fp.write(line(key, *tokenized[key]))